##**`3. ▶ Data Processing :`**

1. Reassigned IDs after cleaning.
2. Removing Diacritization (التشكيل) and punctuation.
3. Replacing links, numbers , dates and mentions with (“رقم”،“تاريخ”، “رابط”،”بريد”) .
4. Converting emojis to equivalent text. (😂 -> face_tearing_with_joy)
5. Normalizing letters. أ إ آ -> ا
6. Lemmatization using multiple tools (camel or farasa) .
7. Removing stopwords. (combined nltk and Arabic-Stopwords)
8. Removing duplicate rows.
9. Tokenization using camel-tools simple word tokenizer.
10. Translate English text ( emojis ) then delete others .
11. Remove numbers or convert to word representation ( date column ).


In [ ]:

# convert to word representation ( date column )
def convert_date_to_arabic(date_str):
    try:
        date = pd.to_datetime(date_str, errors='coerce')
        if pd.isna(date):
            return ""
        day = date.day
        month = date.month
        year = date.year
        day_word = num2words(day, lang='ar')

        months_ar = [
            'يناير', 'فبراير', 'مارس', 'أبريل', 'مايو', 'يونيو',
            'يوليو', 'أغسطس', 'سبتمبر', 'أكتوبر', 'نوفمبر', 'ديسمبر'
        ]
        month_word = months_ar[month - 1]
        year_word = num2words(year, lang='ar')

        return f" تاريخ نشر الخبر  {day_word} من شهر {month_word} سنة {year_word}"

    except Exception as e:
        print(f"Error converting date '{date_str}': {e}")
        return ""

# Apply at date column
real_fake_news['date'] = real_fake_news['date'].apply(convert_date_to_arabic)

real_fake_news

,Id,date,platform,title,News content,Label
0,1,تاريخ نشر الخبر أحد عشر من شهر يناير سنة ألف...,Aljazeera,الضفة الغربية.. الاحتلال يهدم 17 منزلا تاريخيا...,هدمت قوات الاحتلال الإسرائيلي -اليوم الأربعاء-...,real
1,2,تاريخ نشر الخبر أحد عشر من شهر يناير سنة ألف...,Aljazeera,مظاهرات بمدن أوروبية تضامنا مع غزة وحشود أمام ...,خرجت مظاهرات في عدد من المدن الأوروبية مساء ال...,real
2,3,تاريخ نشر الخبر أحد عشر من شهر يناير سنة ألف...,Aljazeera,شهداء في جنين وطولكرم وإضراب عام بالضفة الغربي...,استشهد 4 فلسطينيين واعتقل عشرات آخرون -اليوم ا...,real
3,4,تاريخ نشر الخبر أحد عشر من شهر فبراير سنة أل...,Aljazeera,أبو عبيدة: خسائر العدو أكبر بكثير مما يعلن وسن...,أكد الناطق باسمكتائب الشهيد عز الدين القسام-ال...,real
4,5,تاريخ نشر الخبر أحد عشر من شهر مارس سنة ألفا...,Aljazeera,9 شهداء بالضفة والاحتلال يشن حملة اعتقالات,استشهد 9 فلسطينيين في مواجهات اندلعت مع قوات ا...,real
...,...,...,...,...,...,...
5345,5346,تاريخ نشر الخبر ثلاثون من شهر مايو سنة ألفان...,Aljazeera,"الحرب على غزة مباشر.. مجازر إسرائيلية بمراكز ""...",في اليوم الـ77 من استئناف حرب الإبادة على غزة،...,real
5346,5347,تاريخ نشر الخبر ثلاثون من شهر مايو سنة ألفان...,Misbar,الصورة ليست لجندي مغربي سقط في كمين للمقاومة ا...,تتداول حسابات على موقع التواصل الاجتماعي إكس، ...,fake
5347,5348,تاريخ نشر الخبر ثلاثون من شهر مايو سنة ألفان...,Aljazeera,مظاهرات بإسرائيل تطالب بإكمال الصفقة وجدل حول ...,تظاهر محتجون وسط تل أبيب ليل الاثنين وأضرموا ا...,real
5348,5349,تاريخ نشر الخبر ثلاثون من شهر مايو سنة ألفان...,Aljazeera,"زكريا الزبيدي حرا.. ""التنين"" الفلسطيني الذي هز...",لم يكن في مخيلة وزير الأمن القومي الإسرائيلي ا...,real


In [ ]:

# Check if Farasa is available
FARASA_AVAILABLE = True
except ImportError:
    FARASA_AVAILABLE = False

# Create a Preprocess class to apply preprocessing steps to data
class Preprocess:
    # Load Arabic stopwords from NLTK and another library
    nltk_arb_stopwords = set(nltk.corpus.stopwords.words("arabic"))
    arabicstopwords = set(stp.stopwords_list())

    def __init__(self, EMOJIS='text', HASH_FREQ=2, BERT=False, use_farasa=True, use_camel=False):
        # Set configuration options
        print("BERT Mode:", BERT)
        print("Emoji handling mode:", EMOJIS)
        print("Using Farasa:", use_farasa and FARASA_AVAILABLE)
        print("Using Camel Tools:", use_camel)

        self.EMOJIS = EMOJIS
        self.HASH_FREQ = HASH_FREQ
        self.BERT = BERT
        self.use_farasa = use_farasa and FARASA_AVAILABLE
        self.use_camel = use_camel
        self.STOPWORDS = self.arabicstopwords.union(['', ' '])
        self.emoji_translation_cache = {}

        # Initialize Farasa stemmer
        if self.use_farasa:
            self.farasa_stm = FarasaStemmer(interactive=True)
        else:
            self.farasa_stm = None

        # Initialize Camel Tools
        if self.use_camel:
            self.db = MorphologyDB.builtin_db()
            self.analyzer = Analyzer(self.db)
            self.mle = MLEDisambiguator.pretrained()

        # Google Translator for emoji translation
        self.translator = Translator()

    # Remove diacritics from Arabic text
    def dediac(self, text):
        return camel_utils_dediac.dediac_ar(text)

    # Clean text from links, numbers, emails, and non-Arabic chars
    def tokens(self, text):
        text = re.sub(r'\b\d{1,4}[-/\.]\d{1,2}[-/\.]\d{1,4}\b', 'تاريخ', text)
        text = re.sub(r'(https?://\S+|www\.\S+)', ' رابط ', text)
        text = re.sub(r'\S*@\S*', ' بريد ', text)
        text = re.sub(r'\d+', ' رقم ', text)
        text = re.sub(r'[^\u0600-\u06FF\s_]', '', text)
        return text

    # Convert emojis to Arabic words using Google Translate
    def convert_emojis_to_meaning(self, text):
        if not isinstance(text, str):
            return text
        if self.EMOJIS == 'text':
            text = emoji.demojize(text, delimiters=(" <", "> "))
            emoji_names = set(re.findall(r'<(.*?)>', text))
            for eng_name in emoji_names:
                if eng_name in self.emoji_translation_cache:
                    ar_translation = self.emoji_translation_cache[eng_name]
                else:
                    try:
                        ar_translation = self.translator.translate(eng_name.replace('_', ' '), src='en', dest='ar').text
                        ar_translation = "ملصق_" + ar_translation.replace(' ', '_')
                        self.emoji_translation_cache[eng_name] = ar_translation
                    except Exception as e:
                        print(f"Translation failed for '{eng_name}': {e}")
                        continue
                text = text.replace(f"<{eng_name}>", ar_translation)
            return text
        elif self.EMOJIS == 'none':
            return emoji.replace_emoji(text, replace='')
        else:
            return emoji.replace_emoji(text, replace=lambda c: f' {c} ')

    # Remove punctuation and keep special tokens/hashtags
    def remove_punctuation(self, text):
        punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ«»⁉'''
        hashtags = re.findall(r'#\S+', text)
        text = re.sub(r'#\S+', '', text)
        tokens = re.findall(r'\b\w+_\w+\b', text)
        text = re.sub(r'\b\w+_\w+\b', '', text)
        text = text.translate(str.maketrans(punc, ' ' * len(punc)))
        return text + ' ' + ' '.join(tokens) + ' ' + ' '.join(hashtags)

    # Normalize Arabic letters like Alef, Teh Marbuta, etc.
    def normalize(self, text):
        text = camel_utils_normalize.normalize_alef_ar(text)
        text = camel_utils_normalize.normalize_teh_marbuta_ar(text)
        text = camel_utils_normalize.normalize_alef_maksura_ar(text)
        return text.lower()

    # Tokenize text and repeat hashtags based on frequency
    def tokenizer(self, text):
        entities = re.findall(r'\b[\w_]+\b', text)
        hashtags = re.findall(r'#(\S+)', text)
        text = re.sub(r'#\S+', '', text)

        words = []
        for token in entities:
            words.append(token)

        for tag in hashtags:
            words.extend([tag] * self.HASH_FREQ)

        return words

    # Apply Farasa stemmer
    def farasa_lemmatize(self, text):
        if not self.use_farasa:
            return []
        ret = self.farasa_stm.stem(text)
        ret_tokens = self.tokenizer(ret)
        return ret_tokens

    # Apply CamelTools lemmatizer
    def camel_lemmatize(self, text):
        tokens = camel_utils_tokenizer.simple_word_tokenize(text)
        disambig = self.mle.disambiguate(tokens)
        lemmas = [camel_utils_dediac.dediac_ar(d.analyses[0][1]['lex']) for d in disambig]
        return lemmas

    # Remove Arabic stopwords
    def remove_stopwords(self, tokenized_text):
        return [tk for tk in tokenized_text if self.dediac(tk) not in self.STOPWORDS]

    # Remove duplicate tokens
    def deduplicate(self, tokens):
        seen = set()
        deduped = []
        for tok in tokens:
            if tok not in seen:
                deduped.append(tok)
                seen.add(tok)
        return deduped

    # Remove unwanted or meaningless tokens
    def clean_tokens(self, tokens):
        return [t for t in tokens if t and not all(ch in "_-–—" for ch in t)]

    # Merge tokens separated by underscores
    def merge_underscore_tokens(self, tokens):
        merged = []
        i = 0
        while i < len(tokens):
            if tokens[i] != '_':
                current = tokens[i]
                i += 1
                while i + 1 < len(tokens) and tokens[i] == '_' and tokens[i+1] != '_':
                    current += '_' + tokens[i+1]
                    i += 2
                merged.append(current)
            else:
                i += 1
        return merged

    # Apply all preprocessing steps
    def do_all(self, text, debug=False):
        text = self.dediac(text)
        text = self.tokens(text)
        text = self.convert_emojis_to_meaning(text)
        text = self.remove_punctuation(text)
        text = self.normalize(text)

        if self.use_camel:
            tokens = self.camel_lemmatize(text)
        else:
            tokens = self.farasa_lemmatize(text)

        tokens = self.merge_underscore_tokens(tokens)
        tokens = self.remove_stopwords(tokens)
        tokens = self.deduplicate(tokens)
        tokens = self.clean_tokens(tokens)

        return tokens

    # Return cleaned tokens as a final text string
    def get_clean_string(self, text, debug=False):
        tokens = self.do_all(text, debug=debug)
        return ' '.join(tokens)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Make inctance from Prep Class
preprocessor = Preprocess(EMOJIS='text', use_farasa=False, use_camel=True)

# To apply at each
def preprocess(row):
    full_text = f"{row['date']} {row['title']} {row['News content']}"
    return preprocessor.get_clean_string(full_text)

real_fake_news['clean_text'] = real_fake_news.apply(preprocess, axis=1)

# Show the first result
print(real_fake_news['clean_text'].iloc[0])

BERT Mode: False
Emoji handling mode: text
Using Farasa: False
Using Camel Tools: True
تاريخ نشر خبر أحد عشرة يناير سنة فان ثلاث عشرون ضفة ٱحتلال هدم رقم منزل تاريخي مسيرة ندد عدوان غزة قوة إسرائيلي يوم أربعاء قرب بيت لحم ٱنطلق حاشد تنديد قطاعغزهفي منطقة متفرق منالضفه جانب حذر واشنطون أمة متحد ٱعتداء مستوطن مستمر فلسطيني قال حسن بريج مدير مكتب هيئة مقاومة جدار ٱستيطان مدينة جيش قرية شوشحله أشار مشيد حجر قديم عمر زاد عام لفت رفقة مستغل ٱنشغال حرب دائرة قطاع وشوشحله صغير وقع جمع غوستاف عتصيون ٱستيطاني قدس خليل سكن عدد عائلة غالبية تم تهجير ماضي عدة محتل رام الله نابلس أريحا طولكرم رفض إسناد دعم تغطية صحفي لغز شبكة إخباري بريد ٱندلع مواجهة عنيف تل كفر قدوم قلقيلية أطلق رصاص قنبلة صوت شاب إد وقوع إصابة تغطيهصحفيهجنود مركز إعلام فجر ٱقتحم بلدة جنين مقدس كبير مكون عسكري مخيم محور فرقة قناص سطح مبنى مطل داهم مواطن عبث محتوى ٱستشهد ٱعتقل عشرات آخر عملية ٱقتحام جديد نفذ إطار تصعيد إطلاق معركة طوفان أقصى شهد إضراب مجزرة غضون وصف ولاية عنف مزعزع ٱستقرار شكل حض إسرائيل ضبط ناطق ٱسم خارجي أمريكي ما